<a href="https://colab.research.google.com/github/Amandaagambire/Galamsey-Watch/blob/main/Galamsey_Watch_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:

!pip install geemap
!pip install folium
!pip install rasterio
!pip install segmentation-models-pytorch
!pip install pyngrok
!pip install flask
!pip install earthengine-api --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00


In [16]:
from google.colab import drive
import ee
import geemap
import folium
import rasterio
from PIL import Image
import numpy as np
import os
import numpy as np
from google.colab import drive
from skimage.color import rgb2gray
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import torch
import torch.nn as nn
from torchvision import transforms
import shutil
from google.colab.output import eval_js
from pyngrok import ngrok
# from flask import Flask, request, jsonify, render_template




# Mount Google Drive
drive.mount('/content/drive')

#Authenticae then initialise google Earth Engine
ee.Authenticate()
ee.Initialize(project = 'ee-amandaagambire22')
print("Earth Engine Initialized")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Earth Engine Initialized


In [17]:

# Define location around Tarkwa
center = ee.Geometry.Point([-1.6183, 5.3016])
roi = center.buffer(10000)

# Sentinel-2 ImageCollection (updated)
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(roi) \
    .filterDate("2022-01-01", "2024-01-01") \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()

# Compute NDVI and NDWI
ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndwi = s2.normalizedDifference(['B3', 'B11']).rename('NDWI')

# Combine bands
combined = s2.select(['B2', 'B3', 'B4', 'B8', 'B11']).addBands([ndvi, ndwi])

# Create interactive map
Map = geemap.Map(center=[5.2, -2.1], zoom=10)
Map.centerObject(roi, 11)

# Add layers
Map.addLayer(combined, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'True Color')
Map.addLayer(ndvi, {'min': -1, 'max': 1, 'palette': ['white', 'green']}, 'NDVI')
Map.addLayer(ndwi, {'min': -1, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI')
Map


Map(center=[5.301610258872802, -1.6182997142842286], controls=(WidgetControl(options=['position', 'transparent…